In [ ]:

import os
import json
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler

# =============================================================================
# [1] 설정
# =============================================================================
WORK_DIR = "/content/drive/MyDrive/P02_SemanticParsing"
BASE_DIR = f"{WORK_DIR}/nia"
TRAIN_PATH = f"{BASE_DIR}/train.json"
TABLE_PATH = f"{BASE_DIR}/tables.json"
SAVE_DIR = f"{WORK_DIR}/saved_models_final/schema_linker_e5"

MODEL_NAME = "upskyy/e5-base-korean"
BATCH_SIZE = 32
EPOCHS = 3
LR = 2e-5
MAX_LEN = 128
TEMPERATURE = 0.05

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔥 Schema Linking 학습 재시작 (Fixed) (Device: {device})")

# =============================================================================
# [2] 데이터 전처리 (버그 수정됨 ★)
# =============================================================================
def load_data(train_path, table_path):
    with open(train_path, 'r') as f:
        train_data = json.load(f)
    with open(table_path, 'r') as f:
        table_list = json.load(f)
        tables = {t['db_id']: t for t in table_list}
    return train_data, tables

def extract_gold_columns(sql_dict):
    """
    Spider SQL 포맷의 중첩 구조를 안전하게 파싱하여 사용된 컬럼 인덱스를 추출합니다.
    """
    used_cols = set()

    # Helper: val_unit에서 컬럼 추출 ([unit_op, col_unit1, col_unit2])
    def get_col_from_val_unit(val_unit):
        if not val_unit: return
        # col_unit1: [agg, col_id, is_distinct]
        col_unit1 = val_unit[1]
        if col_unit1: used_cols.add(col_unit1[1])
        col_unit2 = val_unit[2]
        if col_unit2: used_cols.add(col_unit2[1])

    # 1. SELECT 절: [is_distinct, [[agg, val_unit], ...]]
    if 'select' in sql_dict:
        select_info = sql_dict['select']
        # index 0은 bool(distinct), index 1이 실제 리스트
        if len(select_info) > 1 and isinstance(select_info[1], list):
            for agg, val_unit in select_info[1]:
                get_col_from_val_unit(val_unit)

    # 2. WHERE 절: [[not, op, val_unit, val1, val2], ...] (조건들의 리스트)
    if 'where' in sql_dict:
        for cond in sql_dict['where']:
            # 조건절은 보통 리스트 형태이며 길이가 3 이상 (AND/OR 등 연결어 제외)
            if isinstance(cond, list) and len(cond) > 2:
                # index 2가 val_unit
                get_col_from_val_unit(cond[2])

    # 3. GROUP BY: [[col_unit...], ...]
    if 'groupBy' in sql_dict and sql_dict['groupBy']:
        for col_unit in sql_dict['groupBy']:
            if col_unit: used_cols.add(col_unit[1])

    # 4. ORDER BY: [order, [val_unit, ...]]
    if 'orderBy' in sql_dict and sql_dict['orderBy']:
        order_info = sql_dict['orderBy']
        if len(order_info) > 1 and isinstance(order_info[1], list):
            for val_unit in order_info[1]:
                get_col_from_val_unit(val_unit)

    # 5. HAVING
    if 'having' in sql_dict and sql_dict['having']:
        for cond in sql_dict['having']:
            if isinstance(cond, list) and len(cond) > 2:
                get_col_from_val_unit(cond[2])

    # 0번 인덱스(*) 제외
    if 0 in used_cols: used_cols.remove(0)

    return list(used_cols)

# =============================================================================
# [3] 데이터셋
# =============================================================================
class SchemaLinkingDataset(Dataset):
    def __init__(self, train_data, tables, tokenizer):
        self.tokenizer = tokenizer
        self.pairs = []

        print("📂 학습 데이터 쌍 생성 중...")
        for item in tqdm(train_data):
            db_id = item['db_id']
            question = item['question']

            if db_id not in tables: continue
            table_info = tables[db_id]

            # 버그 수정된 함수로 정답 컬럼 추출
            gold_col_indices = extract_gold_columns(item['sql'])

            if not gold_col_indices: continue # 정답 컬럼이 없으면 스킵 (*만 있는 경우 등)

            for idx in gold_col_indices:
                # 테이블/컬럼 정보 안전하게 가져오기
                t_names = table_info.get('table_names_original', ["UNKNOWN"])
                t_name_en = t_names[0] if isinstance(t_names, list) else str(t_names)

                c_names_en = table_info['column_names_original']
                c_names_ko = table_info['column_names']

                # 인덱스 범위 체크 (데이터셋 오류 방지)
                if idx >= len(c_names_en): continue

                c_name_en_str = c_names_en[idx][1]
                c_name_ko_str = c_names_ko[idx][1]

                # "query: ..." / "passage: TB_NAME.COL_NAME(한글명)"
                q_text = f"query: {question}"
                p_text = f"passage: {t_name_en}.{c_name_en_str}({c_name_ko_str})"

                self.pairs.append((q_text, p_text))

        print(f"✅ 총 {len(self.pairs)}개의 (질문, 정답컬럼) 데이터 생성 완료!")

    def __len__(self): return len(self.pairs)
    def __getitem__(self, idx): return self.pairs[idx]

def collate_fn(batch):
    queries = [b[0] for b in batch]
    passages = [b[1] for b in batch]
    tok_q = tokenizer(queries, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    tok_p = tokenizer(passages, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    return tok_q, tok_p

# =============================================================================
# [4] 모델 & 학습
# =============================================================================
class E5Model(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(model_name)
    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden = outputs.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
        sum_embeddings = torch.sum(last_hidden * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        embeddings = sum_embeddings / sum_mask
        return torch.nn.functional.normalize(embeddings, p=2, dim=1)

if __name__ == '__main__':
    train_data, tables = load_data(TRAIN_PATH, TABLE_PATH)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # 데이터셋 생성 (수정된 로직 적용)
    dataset = SchemaLinkingDataset(train_data, tables, tokenizer)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

    model = E5Model(MODEL_NAME).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss()
    scaler = GradScaler()

    print("\n🚀 학습 시작...")
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        loop = tqdm(loader, desc=f"Epoch {epoch+1}")

        for tok_q, tok_p in loop:
            tok_q = {k: v.to(device) for k, v in tok_q.items()}
            tok_p = {k: v.to(device) for k, v in tok_p.items()}

            optimizer.zero_grad()
            with autocast():
                q_emb = model(tok_q['input_ids'], tok_q['attention_mask'])
                p_emb = model(tok_p['input_ids'], tok_p['attention_mask'])
                sim_scores = torch.matmul(q_emb, p_emb.T) / TEMPERATURE
                labels = torch.arange(sim_scores.size(0)).to(device)
                loss = loss_fn(sim_scores, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
            loop.set_postfix(loss=f"{loss.item():.4f}")

        save_path = f"{SAVE_DIR}/epoch_{epoch+1}"
        os.makedirs(save_path, exist_ok=True)
        model.backbone.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✨ Epoch {epoch+1} 저장 완료")

    print("🎉 학습 완료! 이제 Top-5 컬럼 검색이 가능합니다.")